In [1]:
import mlflow.onnx
import onnx
model = onnx.load("/home/eternal/model_repository/test/1/bert_model.onnx")
mlflow.onnx.log_model(model, "triton", registered_model_name="bert_model")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023/12/10 21:25:30 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: No module named 'smmap'
Registered model 'bert_model' already exists. Creating a new version of this model...
Created version '2' of model 'bert_model'.


In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader
# 한글 자연어 처리 데이터셋
from Korpora import Korpora
import pandas as pd
from transformers import BertTokenizerFast, BertModel
from tqdm import tqdm  # Progress Bar 출력
from transformers import BertConfig
from transformers import BertModel
import math
import time
import torch.onnx

# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

corpus = Korpora.load("nsmc")

train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))
train = train.loc[train['length'] > 5]
# 전체 데이터셋 크기가 커서 1000개의 문장을 샘플링 합니다.
train = train.sample(1000)
test = test.loc[test['length'] > 5]
test = test.sample(500)

CHECKPOINT_NAME = 'kykim/bert-kor-base'

class TokenDataset(Dataset):
  
    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe        
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)


# 토크나이저 지정
tokenizer_pretrained = CHECKPOINT_NAME
# train, test 데이터셋 생성
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)


tokenizer_bert = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model_bert = BertModel.from_pretrained("kykim/bert-kor-base")

# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)


inputs.keys()

inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape


config = BertConfig.from_pretrained(CHECKPOINT_NAME)

inputs.keys()

inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape


config = BertConfig.from_pretrained(CHECKPOINT_NAME)
config

labels

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)
model_bert


output = model_bert(**inputs)
output.keys()
output['last_hidden_state'].shape, output['pooler_output'].shape
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])


pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

fc = nn.Linear(768, 2)

fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))


class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        start = time.time()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        end = time.time()
           
        return x

    
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)
loss_fn = nn.CrossEntropyLoss()
# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(bert.parameters(), lr=1e-5)
#--------------------------------------------------------------------------------


def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()
    
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0
    
    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    
    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)
        
        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()
        
        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)
        
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)
        
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()
        
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()
        
        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)
        
        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(labels).sum().item()
        counts += len(labels)
        
        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)
        
        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)
    
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc


def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다. 
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()
    
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0
        
        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)
            
            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)
            
            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(labels)).item()
            
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)
        
        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)
        
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

num_epochs = 100

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'bert-kor-base'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)   
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        #dummy_input = torch.randn(1, 50000)
        #torch.onnx.export(bert, input_ids, attention_mask, token_type_ids,'model.onnx')
        torch.save(bert.state_dict(), f'{model_name}.pth')
    

 #   print(f"{end - start:.5f} sec")
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')
bert.load_state_dict(torch.load(f'{model_name}.pth'))
import time#-------------------------------------------------------------------------------------------------------------
'''class CustomPredictor():
   

    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels
        
    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
 
           add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

labels = {
    0: '부정 리뷰 입니다.', 
    1: '긍정 리뷰 입니다.'
}

predictor = CustomPredictor(bert, tokenizer, labels)
'''



def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)

predict_sentence(predictor) 


사용 디바이스: cuda:1

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /home/eternal/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed 

training loss: 5.38427, training accuracy: 0.62200: 100%|█| 125/125 [00:14<00:00


[INFO] val_loss has been improved from inf to 0.47317. Saving Model!
epoch 01, loss: 0.67303, acc: 0.62200, val_loss: 0.47317, val_accuracy: 0.79000


training loss: 3.13939, training accuracy: 0.83300: 100%|█| 125/125 [00:15<00:00


[INFO] val_loss has been improved from 0.47317 to 0.39540. Saving Model!
epoch 02, loss: 0.39242, acc: 0.83300, val_loss: 0.39540, val_accuracy: 0.81800


...

training loss: 0.00014, training accuracy: 1.00000: 100%|█| 125/125 [00:14<00:00


epoch 99, loss: 0.00002, acc: 1.00000, val_loss: 1.61631, val_accuracy: 0.82200


training loss: 0.40007, training accuracy: 0.99100: 100%|█| 125/125 [00:14<00:00


epoch 100, loss: 0.05001, acc: 0.99100, val_loss: 0.90668, val_accuracy: 0.81800


NameError: name 'tokenizer' is not defined

In [2]:
import time

class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels
        
    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)

        # 추론 시작 시간 기록
        start_time = time.time()

        # 모델 추론 수행
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        
        # 추론 종료 시간 기록
        end_time = time.time()

        # 추론에 걸린 시간 계산 (초 단위)
        inference_time = end_time - start_time

        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')
        print(f"추론 시간: {inference_time:.3f}초")

# 이후의 코드 (tokenizer 및 labels 정의, predictor 인스턴스 생성 등)는 동일하게 유지합니다.
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

labels = {
    0: '부정 리뷰 입니다.', 
    1: '긍정 리뷰 입니다.'
}

predictor = CustomPredictor(bert, tokenizer, labels)




def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)

predict_sentence(predictor) 

문장을 입력해 주세요: 야
[부정 리뷰 입니다.]
확률은: 84.699% 입니다.
추론 시간: 0.005초


In [1]:
import torch
import torch.nn as nn
from transformers import BertTokenizerFast, BertModel

class SentimentClassifier(nn.Module):
    def __init__(self, bert_pretrained, num_classes=2, dropout_rate=0.1):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_pretrained)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = output['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

# 토크나이저 및 모델 초기화
tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model = SentimentClassifier("kykim/bert-kor-base")

# 입력 예시
input_text = "영화 정말 좋아요!"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
attention_mask = torch.ones_like(input_ids)
token_type_ids = torch.zeros_like(input_ids)

# 모델 추론
output = model(input_ids, attention_mask, token_type_ids)
print("Model Output:", output)


# ONNX로 변환
onnx_path = "sentiment_classifier.onnx"
dummy_input = (input_ids, attention_mask, token_type_ids)
torch.onnx.export(model, dummy_input, onnx_path, verbose=True,
                  input_names=['input_ids', 'attention_mask', 'token_type_ids'],
                  output_names=['output'],
                  dynamic_axes={'input_ids': [0], 'attention_mask': [0], 'token_type_ids': [0], 'output': [0]})
#print(test)
#print(train)
print(input_ids)
print(output)


Model Output: tensor([[-0.1577, -0.0783]], grad_fn=<AddmmBackward0>)


/home/eternal/.local/lib/python3.10/site-packages/torch/onnx/utils.py:2082: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input_ids
  warnings.warn(
/home/eternal/.local/lib/python3.10/site-packages/torch/onnx/utils.py:2082: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input attention_mask
  warnings.warn(
/home/eternal/.local/lib/python3.10/site-packages/torch/onnx/utils.py:2082: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input token_type_ids
  warnings.warn(
/home/eternal/.local/lib/python3.10/site-packages/torch/onnx/utils.py:2082: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axe

Exported graph: graph(%input_ids : Long(*, 6, strides=[6, 1], requires_grad=0, device=cpu),
      %attention_mask : Long(*, 6, strides=[6, 1], requires_grad=0, device=cpu),
      %token_type_ids : Long(*, 6, strides=[6, 1], requires_grad=0, device=cpu),
      %bert.embeddings.word_embeddings.weight : Float(42000, 768, strides=[768, 1], requires_grad=1, device=cpu),
      %bert.embeddings.position_embeddings.weight : Float(512, 768, strides=[768, 1], requires_grad=1, device=cpu),
      %bert.embeddings.token_type_embeddings.weight : Float(2, 768, strides=[768, 1], requires_grad=1, device=cpu),
      %bert.embeddings.LayerNorm.weight : Float(768, strides=[1], requires_grad=1, device=cpu),
      %bert.embeddings.LayerNorm.bias : Float(768, strides=[1], requires_grad=1, device=cpu),
      %bert.encoder.layer.0.attention.self.query.bias : Float(768, strides=[1], requires_grad=1, device=cpu),
      %bert.encoder.layer.0.attention.self.key.bias : Float(768, strides=[1], requires_grad=1, devic

KeyboardInterrupt: 

In [2]:
import onnx
import onnxruntime
import torch

# 예제로 사용한 ONNX 파일의 경로
onnx_path = "bert_model.onnx"

# ONNX 파일 로드
onnx_model = onnx.load(onnx_path)

# ONNX Runtime 세션 생성
ort_session = onnxruntime.InferenceSession(onnx_path)

# 입력 데이터 생성 (실제 데이터를 사용하도록 수정 필요)
sample_input = {
    'input_ids': torch.ones((1, 128), dtype=torch.long).numpy(),
    'attention_mask': torch.ones((1, 128), dtype=torch.long).numpy(),
    'token_type_ids': torch.ones((1, 128), dtype=torch.long).numpy(),
}

# ONNX Runtime을 사용하여 추론 실행
ort_inputs = {ort_session.get_inputs()[i].name: sample_input[name] for i, name in enumerate(['input_ids', 'attention_mask', 'token_type_ids'])}
ort_outputs = ort_session.run(None, ort_inputs)

# 추론 결과 출력
print("Output shape:", ort_outputs[0].shape)
print("Output values:", ort_outputs[0])

Output shape: (1, 2)
Output values: [[-0.35465354  0.894657  ]]


In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader
# 한글 자연어 처리 데이터셋
from Korpora import Korpora
import pandas as pd
from transformers import BertTokenizerFast, BertModel
from tqdm import tqdm  # Progress Bar 출력
from transformers import BertConfig
from transformers import BertModel
import math
import time
import torch.onnx

# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

corpus = Korpora.load("nsmc")

train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))
train = train.loc[train['length'] > 5]
# 전체 데이터셋 크기가 커서 1000개의 문장을 샘플링 합니다.
train = train.sample(1000)
test = test.loc[test['length'] > 5]
test = test.sample(500)

CHECKPOINT_NAME = 'kykim/bert-kor-base'

class TokenDataset(Dataset):
  
    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe        
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)


# 토크나이저 지정
tokenizer_pretrained = CHECKPOINT_NAME
# train, test 데이터셋 생성
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)


tokenizer_bert = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model_bert = BertModel.from_pretrained("kykim/bert-kor-base")

# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)


inputs.keys()

inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape


config = BertConfig.from_pretrained(CHECKPOINT_NAME)

inputs.keys()

inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape


config = BertConfig.from_pretrained(CHECKPOINT_NAME)
config

labels

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)
model_bert


output = model_bert(**inputs)
output.keys()
output['last_hidden_state'].shape, output['pooler_output'].shape
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])


pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

fc = nn.Linear(768, 2)

fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))


class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        start = time.time()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        end = time.time()
           
        return x

    
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)
loss_fn = nn.CrossEntropyLoss()
# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

#------------------------------------------------------------------------------------------------------------------------------
import torch.onnx
import onnx

# 모델과 토크나이저 정의
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)
model = CustomBertModel(CHECKPOINT_NAME)

# 예제 입력 데이터 생성 (배치 크기 1)
example_input = {
    'input_ids': torch.zeros(1, 512, dtype=torch.long),
    'attention_mask': torch.ones(1, 512, dtype=torch.long),
    'token_type_ids': torch.zeros(1, 512, dtype=torch.long)
}

# 모델을 evaluation 모드로 설정
model.eval()

# 모델의 forward 함수에 대한 정보를 제공하여 ONNX로 변환
onnx_filename = "custom_bert_model.onnx"
torch.onnx.export(
    model,
    (example_input['input_ids'], example_input['attention_mask'], example_input['token_type_ids']),
    onnx_filename,
    input_names=["input_ids", "attention_mask", "token_type_ids"],
    output_names=["output"],
    dynamic_axes={"input_ids": {0: "batch_size"}, "attention_mask": {0: "batch_size"}, "token_type_ids": {0: "batch_size"}, "output": {0: "batch_size"}}
)

# ONNX 모델 로드
onnx_model = onnx.load(onnx_filename)

# ONNX 모델의 구조 출력
print(onnx_model)

# 모델의 input shape 확인
print("Input shape:", onnx_model.graph.input[0].type.tensor_type.shape.dim)









'''
dummy_input = (
    inputs['input_ids'],
    inputs['attention_mask'],
    inputs['token_type_ids']
)
onnx_path = "custom_bert_models.onnx"
torch.onnx.export(bert, dummy_input, onnx_path, input_names=['input_ids', 'attention_mask', 'token_type_ids'])

print(f"ONNX 모델이 {onnx_path} 경로에 저장되었습니다.")
'''


사용 디바이스: cuda:1

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /home/eternal/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed 

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacty of 23.63 GiB of which 107.44 MiB is free. Process 17985 has 8.68 GiB memory in use. Process 99943 has 11.03 GiB memory in use. Including non-PyTorch memory, this process has 3.34 GiB memory in use. Of the allocated memory 2.83 GiB is allocated by PyTorch, and 67.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# load model and tokenizer
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
dummy_model_input = tokenizer("This is a sample", return_tensors="pt")

# export
torch.onnx.export(
    model, 
    tuple(dummy_model_input.values()),
    f="torch-model.onnx",  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    do_constant_folding=True, 
    opset_version=13, 
)



/home/eternal/.local/lib/python3.10/site-packages/transformers/models/distilbert/modeling_distilbert.py:245: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [5]:
from pathlib import Path
import transformers
from transformers.onnx import FeaturesManager
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

# load model and tokenizer
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
feature = "sequence-classification"
model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# load config
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature=feature)
onnx_config = model_onnx_config(model.config)

# export
onnx_inputs, onnx_outputs = transformers.onnx.export(
        preprocessor=tokenizer,
        model=model,
        config=onnx_config,
        opset=13,
        output=Path("trfs-model.onnx")
)


2023-12-18 19:05:37.679194: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-18 19:05:37.700760: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 19:05:37.826914: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 19:05:37.826970: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 19:05:37.827777: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [6]:
torch_input='나아가 끝나는 모시깽이'

In [ ]:
import onnx
onnx_model = onnx.load("bert_model.onnx")
onnx.checker.check_model(onnx_model)
onnx_program = torch.onnx.dynamo_export(onnx_model, torch_input)

/home/eternal/.local/lib/python3.10/site-packages/torch/onnx/_internal/exporter.py:130: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [23]:
import onnxruntime

onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

ort_session = onnxruntime.InferenceSession("./bert_model.onnx", providers=['CPUExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

NameError: name 'onnx_program' is not defined

In [24]:
batch_size = 8
dummy_input_ids = torch.randint(0, 100, (batch_size, 512)).to(device)
dummy_attention_mask = torch.randint(0, 2, (batch_size, 512)).to(device)
dummy_token_type_ids = torch.randint(0, 2, (batch_size, 512)).to(device)

# 모델 인스턴스를 생성하고 device로 옮깁니다.
bert_model = CustomBertModel(CHECKPOINT_NAME).to(device)

# 모델을 평가 모드로 변경
bert_model.eval()

# 입력값에 대한 예측을 수행
with torch.no_grad():
    onnx_output = bert_model(dummy_input_ids, dummy_attention_mask, dummy_token_type_ids)

# ONNX로 변환할 때 사용할 입력값 정의
onnx_input = (dummy_input_ids, dummy_attention_mask, dummy_token_type_ids)

# ONNX로 모델을 변환
onnx_path = "custom_bert_model.onnx"
torch.onnx.export(
    bert_model,
    onnx_input,
    onnx_path,
    input_names=['input_ids', 'attention_mask', 'token_type_ids'],
    output_names=['output'],
    opset_version=11,
    dynamic_axes={'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'token_type_ids': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

print(f"ONNX 모델이 {onnx_path}로 성공적으로 저장되었습니다.")

ONNX 모델이 custom_bert_model.onnx로 성공적으로 저장되었습니다.


In [6]:
import onnx
import onnxruntime


# 저장된 ONNX 모델을 불러오기
onnx_model = onnx.load(onnx_path)

# ONNX Runtime으로 모델 실행
ort_session = onnxruntime.InferenceSession(onnx_path)

# 입력값 생성
ort_inputs = {
    'input_ids': dummy_input_ids.cpu().numpy(),
    'attention_mask': dummy_attention_mask.cpu().numpy(),
    'token_type_ids': dummy_token_type_ids.cpu().numpy()
}

# ONNX Runtime으로 모델 실행
ort_outs = ort_session.run(None, ort_inputs)
print("ONNX Runtime으로 실행된 결과:", ort_outs)

NameError: name 'onnx_path' is not defined

In [7]:
import onnx

# ONNX 모델 로드
onnx_model_path = "custom_bert_model.onnx"
onnx_model = onnx.load(onnx_model_path)

# 입력 및 출력 이름 및 모양 확인
input_names = [input.name for input in onnx_model.graph.input]
output_names = [output.name for output in onnx_model.graph.output]
input_shapes = {input.name: input.type.tensor_type.shape.dim for input in onnx_model.graph.input}
output_shapes = {output.name: output.type.tensor_type.shape.dim for output in onnx_model.graph.output}

print("Input Names:", input_names)
print("Output Names:", output_names)
print("Input Shapes:", input_shapes)
print("Output Shapes:", output_shapes)

Input Names: ['input_ids', 'attention_mask', 'token_type_ids']
Output Names: ['output']
Input Shapes: {'input_ids': [dim_param: "batch_size"
, dim_value: 512
], 'attention_mask': [dim_param: "batch_size"
, dim_value: 512
], 'token_type_ids': [dim_param: "batch_size"
, dim_value: 512
]}
Output Shapes: {'output': [dim_param: "batch_size"
, dim_value: 2
]}


In [8]:
import onnxruntime
import numpy as np

# ONNX 모델 로드
onnx_model_path = "custom_bert_model.onnx"
ort_session = onnxruntime.InferenceSession(onnx_model_path)

# 입력 데이터 생성 (예시)
input_ids = np.random.randint(0, 100, (batch_size, 512)).astype(np.int64)
attention_mask = np.random.randint(0, 2, (batch_size, 512)).astype(np.int64)
token_type_ids = np.random.randint(0, 2, (batch_size, 512)).astype(np.int64)

# ONNX Runtime으로 추론 수행
ort_inputs = {'input_ids': input_ids, 'attention_mask': attention_mask, 'token_type_ids': token_type_ids}
ort_outs = ort_session.run(None, ort_inputs)

# ONNX 결과 확인
print("ONNX Runtime으로 실행된 결과:", ort_outs)

NameError: name 'batch_size' is not defined

In [9]:
onnx_predictions = np.argmax(ort_outs[0], axis=1)

# 정확도 계산
onnx_accuracy = np.sum(onnx_predictions == labels.numpy()) / len(labels)
print(f'ONNX Accuracy: {onnx_accuracy * 100:.2f}%')

NameError: name 'labels' is not defined

In [4]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
text = "안녕하세요, 테스트 문장입니다."
tokens = tokenizer(text, return_tensors="pt")


In [5]:
import onnxruntime
import numpy as np
from transformers import BertTokenizerFast

# 텍스트 입력
text = "안녕하세요, 테스트 문장입니다."
max_length = 512  # BERT의 최대 입력 길이

# BERT 토크나이저 로드
tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")

# 텍스트를 토큰화하고 입력 형태로 변환
tokens = tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True, padding="max_length")
input_ids = np.array(tokens["input_ids"]).astype(np.int64)
attention_mask = np.array(tokens["attention_mask"]).astype(np.int64)
token_type_ids = np.array(tokens["token_type_ids"]).astype(np.int64)

# ONNX 모델 로드
onnx_model_path = "custom_bert_model.onnx"
ort_session = onnxruntime.InferenceSession(onnx_model_path)

# 입력 데이터의 차원을 모델이 기대하는 형태로 맞추기
ort_inputs = {
    'input_ids': input_ids.reshape(1, -1),
    'attention_mask': attention_mask.reshape(1, -1),
    'token_type_ids': token_type_ids.reshape(1, -1)
}

# ONNX Runtime으로 추론 수행
ort_outs = ort_session.run(None, ort_inputs)

# ONNX 결과 확인
print("ONNX Runtime으로 실행된 결과:", ort_outs)

ONNX Runtime으로 실행된 결과: [array([[ 0.5840088, -0.3010565]], dtype=float32)]


In [50]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader
# 한글 자연어 처리 데이터셋
from Korpora import Korpora
import pandas as pd
from transformers import BertTokenizerFast, BertModel
from tqdm import tqdm  # Progress Bar 출력
from transformers import BertConfig
from transformers import BertModel
import math
import time
import torch.onnx

# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

corpus = Korpora.load("nsmc")

train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))
train = train.loc[train['length'] > 5]
# 전체 데이터셋 크기가 커서 1000개의 문장을 샘플링 합니다.
train = train.sample(1000)
test = test.loc[test['length'] > 5]
test = test.sample(500)

CHECKPOINT_NAME = 'kykim/bert-kor-base'

class TokenDataset(Dataset):
  
    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe        
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)


# 토크나이저 지정
tokenizer_pretrained = CHECKPOINT_NAME
# train, test 데이터셋 생성
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)


tokenizer_bert = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model_bert = BertModel.from_pretrained("kykim/bert-kor-base")

# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)


inputs.keys()

inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape


config = BertConfig.from_pretrained(CHECKPOINT_NAME)

inputs.keys()

inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape


config = BertConfig.from_pretrained(CHECKPOINT_NAME)
config

labels

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)
model_bert


output = model_bert(**inputs)
output.keys()
output['last_hidden_state'].shape, output['pooler_output'].shape
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])


pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

fc = nn.Linear(768, 2)

fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))

'''
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        start = time.time()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        end = time.time()
           
        return x
'''

class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        print("last_hidden_state shape:", last_hidden_state.shape)
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        print("Output shape:", x.shape)
           
        return x

# 모델 생성
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)

# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

# forward 수행
outputs = bert(**inputs)



bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)
loss_fn = nn.CrossEntropyLoss()
# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)
optimizer = optim.Adam(bert.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

# 학습 루프
epochs = 3
for epoch in range(epochs):
    for batch in train_loader:
        inputs, labels = batch
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # Forward pass
        outputs = bert(**inputs)
        loss = loss_fn(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# 학습된 모델 저장
torch.save(bert.state_dict(), 'fine_tuned_bert.pth')

import torch
from transformers import BertTokenizerFast, BertModel

# 모델 및 토크나이저 불러오기
model_checkpoint = "kykim/bert-kor-base"
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
model = BertModel.from_pretrained(model_checkpoint)

# 예제 입력 데이터 생성
example_input = {
    'input_ids': torch.tensor([tokenizer.encode("예제 문장", max_length=512, padding='longest')]),
    'attention_mask': torch.tensor([tokenizer.encode("예제 문장", max_length=512, padding='longest')]),
    'token_type_ids': torch.tensor([tokenizer.encode("예제 문장", max_length=512, padding='longest')]),
}

# 모델을 평가 모드로 설정
model.eval()

# 모델을 ONNX 형식으로 변환
onnx_path = "model.onnx"

# 각 텐서의 차원 확인
print("input_ids shape:", example_input['input_ids'].shape)
print("attention_mask shape:", example_input['attention_mask'].shape)
print("token_type_ids shape:", example_input['token_type_ids'].shape)

# 모델을 ONNX 형식으로 변환
torch.onnx.export(
    model,
    (
        example_input['input_ids'],
        example_input['attention_mask'],
        example_input['token_type_ids'],
    ),
    onnx_path,
    verbose=True,
    input_names=['input_ids', 'attention_mask', 'token_type_ids'],
    output_names=['output']
)

print(f"ONNX 모델이 {onnx_path}에 저장되었습니다.")

'''
import torch
import torch.onnx
from transformers import BertTokenizerFast, BertModel

# 모델 및 토크나이저 불러오기
model_checkpoint = "kykim/bert-kor-base"
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
model = BertModel.from_pretrained(model_checkpoint)

# 예제 입력 데이터 생성
input_ids = torch.tensor([tokenizer.encode("아 재미없다.", max_length=128, padding='longest')])
attention_mask = torch.tensor([tokenizer.encode("아 재미없다.", max_length=128, padding='longest')])

token_type_ids = torch.tensor([tokenizer.encode("아 재미없다.", max_length=128, padding='longest')])

# 모델을 평가 모드로 설정
model.eval()

# 모델을 ONNX 형식으로 변환
onnx_path = "model.onnx"
torch.onnx.export(
    model,
    (input_ids, attention_mask, token_type_ids),
    onnx_path,
    verbose=True,
    input_names=['input_ids', 'attention_mask', 'token_type_ids'],
    output_names=['last_hidden_state', 'pooler_output']
)

print(f"ONNX 모델이 {onnx_path}에 저장되었습니다.")'''

사용 디바이스: cuda:1

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /home/eternal/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed 

last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size(

last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size(

last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size(

last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
last_hidden_state shape: torch.Size([8, 512, 768])
Output shape: torch.Size([8, 2])
input_ids shape: torch.Size([1, 5])
attention_mask shape: torch.Size([1, 5])
token_type_ids shape: torch.Size([1, 5])


IndexError: index out of range in self

In [13]:
import torch.onnx

# Assuming 'bert' is an instance of CustomBertModel
dummy_input = (torch.ones(1, 512).long().to(device),  # input_ids
               torch.ones(1, 512).long().to(device),  # attention_mask
               torch.zeros(1, 512).long().to(device))  # token_type_ids

onnx_path = "custom_bert_model.onnx"
torch.onnx.export(bert, dummy_input, onnx_path, input_names=['input_ids', 'attention_mask', 'token_type_ids'], output_names=['output'])


RuntimeError: Trying to create tensor with negative dimension -1: [1, -1]

In [28]:
#!/usr/bin/python

# Copyright (c) 2023, NVIDIA CORPORATION. All rights reserved. 
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License. 
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, 
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 
# See the License for the specific language governing permissions and
# limitations under the License. 

import torch
import pickle
import argparse
import deployer_lib
# 
import sys
sys.path.append('../')
sys.path.append('.')
from modeling import BertForQuestionAnswering, BertConfig
from tokenization import BertTokenizer
from run_squad import convert_examples_to_features, read_squad_examples


def get_model_args(model_args):
    ''' the arguments initialize_model will receive '''
    parser = argparse.ArgumentParser()
    ## Required parameters by the model. 
    parser.add_argument("--checkpoint", 
                        default=None, 
                        type=str, 
                        required=True, 
                        help="The checkpoint of the model. ")
    parser.add_argument('--batch_size', 
                        default=8, 
                        type=int, 
                        help='Batch size for inference')
    parser.add_argument("--bert_model", default="bert-large-uncased", type=str, 
                        help="Bert pre-trained model selected in the list: bert-base-uncased, "
                             "bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, "
                             "bert-base-multilingual-cased, bert-base-chinese.")
    parser.add_argument("--do_lower_case", 
                        action='store_true', 
                        help="Whether to lower case the input text. True for uncased models, False for cased models.")
    parser.add_argument('--vocab_file', 
                        type=str, default=None, required=True, 
                        help="Vocabulary mapping/file BERT was pretrainined on")
    parser.add_argument("--predict_file", default=None, type=str, 
                        help="SQuAD json for predictions. E.g., dev-v1.1.json or test-v1.1.json")
    parser.add_argument('--version_2_with_negative', 
                        action='store_true', 
                        help='If true, the SQuAD examples contain some that do not have an answer.')
    parser.add_argument("--max_seq_length", default=384, type=int, 
                        help="The maximum total input sequence length after WordPiece tokenization. Sequences "
                             "longer than this will be truncated, and sequences shorter than this will be padded.")
    parser.add_argument("--doc_stride", default=128, type=int, 
                        help="When splitting up a long document into chunks, how much stride to take between chunks.")
    parser.add_argument("--max_query_length", default=64, type=int, 
                        help="The maximum number of tokens for the question. Questions longer than this will " 
                             "be truncated to this length.")
    parser.add_argument("--config_file", 
                        default=None, 
                        type=str, 
                        required=True, 
                        help="The BERT model config")
    parser.add_argument('--fp16',
                        action='store_true',
                        help="use mixed-precision")
    parser.add_argument('--nbatches', 
                        default=2, 
                        type=int, 
                        help='Number of batches in the inference dataloader. Default: 10. ')
    return parser.parse_args(model_args)


def initialize_model(args):
    ''' return model, ready to trace '''
    config = BertConfig.from_json_file(args.config_file)
    if config.vocab_size % 8 != 0:
        config.vocab_size += 8 - (config.vocab_size % 8)
    model = BertForQuestionAnswering(config)
    model.enable_apex(False)
    state_dict = torch.load(args.checkpoint, map_location='cpu')["model"]
    model.load_state_dict(state_dict)
    if args.fp16:
        model.half()
    return model


def get_dataloader(args):
    ''' return dataloader for inference '''
    
    # Preprocess input data
    tokenizer = BertTokenizer(args.vocab_file, do_lower_case=args.do_lower_case, max_len=512) # for bert large
    
    cached_features_file = args.predict_file + '_{}_{}.bin'.format(args.max_seq_length, args.doc_stride)
    try:
        with open(cached_features_file, "rb") as reader:
            eval_features = pickle.load(reader)
    except:
        eval_examples = read_squad_examples(
            input_file=args.predict_file,
            is_training=False,
            version_2_with_negative=args.version_2_with_negative)
        eval_features = convert_examples_to_features(
            examples=eval_examples,
            tokenizer=tokenizer,
            max_seq_length=args.max_seq_length,
            doc_stride=args.doc_stride,
            max_query_length=args.max_query_length,
            is_training=False)
        with open(cached_features_file, "wb") as writer:
            pickle.dump(eval_features, writer)
    
    data = []
    for feature in eval_features:
        input_ids = torch.tensor(feature.input_ids, dtype=torch.int64)
        input_mask = torch.tensor(feature.input_mask, dtype=torch.int64)
        segment_ids = torch.tensor(feature.segment_ids, dtype=torch.int64)
        inp = (input_ids, segment_ids, input_mask)
        data.append(inp)
    
    if args.nbatches > 0:
        data = data[:args.nbatches*args.batch_size]
    
    test_loader = torch.utils.data.DataLoader(
        data, 
        batch_size=args.batch_size, 
        shuffle=False, 
        num_workers=1, 
        pin_memory=True)
    
    return test_loader


if __name__=='__main__':
    # don't touch this!
    deployer, model_argv = deployer_lib.create_deployer(sys.argv[1:]) # deployer and returns removed deployer arguments
    
    model_args = get_model_args(model_argv)
    
    model = initialize_model(model_args)
    dataloader = get_dataloader(model_args)
    
    deployer.deploy(dataloader, model)

ModuleNotFoundError: No module named 'deployer_lib'

In [29]:
!python ./deployer/deployer.py \
    --{exportFormat} \
    --save-dir ./candidatemodels \
    --triton-model-name {modelName} \
    --triton-model-version 1 \
    --triton-max-batch-size 8 \
    --triton-dyn-batching-delay 0 \
    --triton-engine-count 1 \
    -- --checkpoint ./data/bert_qa.pt \
    --config_file ./bert_config.json \
    --vocab_file ./vocab \
    --predict_file ./squad/v1.1/dev-v1.1.json \
    --do_lower_case \
    --batch_size=8

/bin/bash: 줄 1: python: 명령어를 찾을 수 없음


최근꺼(12/19)

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertTokenizerFast, BertModel
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig
from transformers import BertTokenizerFast
from Korpora import Korpora
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# 한글 자연어 처리 데이터셋
from Korpora import Korpora

# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

corpus = Korpora.load("nsmc")
import pandas as pd

train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))
train = train.loc[train['length'] > 5]
# 전체 데이터셋 크기가 커서 1000개의 문장을 샘플링 합니다.
train = train.sample(1000)
test = test.loc[test['length'] > 5]
test = test.sample(500)

CHECKPOINT_NAME = 'kykim/bert-kor-base'

class TokenDataset(Dataset):
    def __init__(self, dataframe, tokenizer_pretrained):
        self.data = dataframe        
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        tokens = self.tokenizer(
            sentence,
            return_tensors='pt',
            truncation=True,
            padding='max_length',
            add_special_tokens=True
        )

        input_ids = tokens['input_ids'].squeeze(0)
        attention_mask = tokens['attention_mask'].squeeze(0)
        token_type_ids = torch.zeros_like(attention_mask)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

tokenizer_pretrained = CHECKPOINT_NAME
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)

tokenizer_bert = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model_bert = BertModel.from_pretrained("kykim/bert-kor-base")

inputs, labels = next(iter(train_loader))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

config = BertConfig.from_pretrained(CHECKPOINT_NAME)

labels

model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)

output = model_bert(**inputs)
output.keys()
output['last_hidden_state'].shape, output['pooler_output'].shape
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])

pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

fc = nn.Linear(768, 2)

fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))

import time

class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        super(CustomBertModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_pretrained)
        self.dr = nn.Dropout(p=dropout_rate)
        self.fc = nn.Linear(768, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = output['last_hidden_state']
        x = self.dr(last_hidden_state[:, 0, :])
        x = self.fc(x)
        return x

bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

from tqdm import tqdm

def model_train(model, data_loader, loss_fn, optimizer, device):
    model.train()
    running_loss = 0
    corr = 0
    counts = 0
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    
    for idx, (inputs, labels) in enumerate(prograss_bar):
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        output = model(**inputs)
        
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        
        _, pred = output.max(dim=1)
        corr += pred.eq(labels).sum().item()
        counts += len(labels)
        
        running_loss += loss.item() * labels.size(0)
        
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        
    acc = corr / len(data_loader.dataset)
    return running_loss / len(data_loader.dataset), acc

def model_evaluate(model, data_loader, loss_fn, device):
    model.eval()
    with torch.no_grad():
        corr = 0
        running_loss = 0
        for inputs, labels in data_loader:
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            
            output = model(**inputs)
            
            _, pred = output.max(dim=1)
            
            corr += torch.sum(pred.eq(labels)).item()
            
            running_loss += loss_fn(output, labels).item() * labels.size(0)
        
        acc = corr / len(data_loader.dataset)
        return running_loss / len(data_loader.dataset), acc

import torch.onnx

num_epochs = 100
model_name = 'bert-kor-base'
min_loss = np.inf

for epoch in range(num_epochs):
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)   
    
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}.pth')
        torch.onnx.export(bert, (inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids']), 'bert-kor-base.onnx', verbose=True)

    
    print(" ") 
    print('Model has been converted to ONNX')
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

bert.load_state_dict(torch.load(f'{model_name}.pth'))

class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels
        
    def predict(self, sentence):
        tokens = self.tokenizer(
            sentence,
            return_tensors='pt',
            truncation=True,
            padding='max_length',
            add_special_tokens=True
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

labels = {
    0: '부정 리뷰 입니다.', 
    1: '긍정 리뷰 입니다.'
}

predictor = CustomPredictor(bert, tokenizer, labels)

def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)

predict_sentence(predictor)

# 모델을 ONNX로 변환
dummy_input = {
    'input_ids': torch.zeros(1, 512).long().to(device),
    'attention_mask': torch.ones(1, 512).long().to(device),
    'token_type_ids': torch.zeros(1, 512).long().to(device)
}
onnx_path = "bert_kor_base.onnx"
torch.onnx.export(bert, (dummy_input['input_ids'], dummy_input['attention_mask'], dummy_input['token_type_ids']), onnx_path, input_names=['input_ids', 'attention_mask', 'token_type_ids'], output_names=['output'], dynamic_axes={'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'token_type_ids': {0: 'batch_size'}})

# Triton Server 설정 파일 생성
triton_config = f"""
name: "bert_kor_base"
platform: "onnxruntime_onnx"

max_batch_size: 1
input [
  {{
    name: "input_ids"
    data_type: TYPE_INT64
    dims: [512]
  }},
  {{
    name: "attention_mask"
    data_type: TYPE_INT64
    dims: [512]
  }},
  {{
    name: "token_type_ids"
    data_type: TYPE_INT64
    dims: [512]
  }}
]
output [
  {{
    name: "output"
    data_type: TYPE_FP32
    dims: [2]
  }}
]
"""

# Triton Server 설정 파일 저장
with open("bert_kor_base_config.pbtxt", "w") as f:
    f.write(triton_config)
#이게 그나마 성공 적인 모시꺵이

사용 디바이스: cuda:1

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /home/eternal/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed 

training loss: 5.42143, training accuracy: 0.62400: 100%|█| 125/125 [00:13<00:00


[INFO] val_loss has been improved from inf to 0.41643. Saving Model!
 
Model has been converted to ONNX
epoch 01, loss: 0.67768, acc: 0.62400, val_loss: 0.41643, val_accuracy: 0.83000


training loss: 3.10725, training accuracy: 0.83400: 100%|█| 125/125 [00:13<00:00


[INFO] val_loss has been improved from 0.41643 to 0.39110. Saving Model!
 
Model has been converted to ONNX
epoch 02, loss: 0.38841, acc: 0.83400, val_loss: 0.39110, val_accuracy: 0.84600


...

training loss: 0.00028, training accuracy: 1.00000: 100%|█| 125/125 [00:13<00:00


 
Model has been converted to ONNX
epoch 99, loss: 0.00004, acc: 1.00000, val_loss: 1.09268, val_accuracy: 0.86400


training loss: 0.00032, training accuracy: 1.00000: 100%|█| 125/125 [00:13<00:00


 
Model has been converted to ONNX
epoch 100, loss: 0.00004, acc: 1.00000, val_loss: 1.09870, val_accuracy: 0.86600
문장을 입력해 주세요: ㅁ0
[부정 리뷰 입니다.]
확률은: 52.214% 입니다.


'# 모델을 ONNX로 변환\ndummy_input = {\n    \'input_ids\': torch.zeros(1, 512).long().to(device),\n    \'attention_mask\': torch.ones(1, 512).long().to(device),\n    \'token_type_ids\': torch.zeros(1, 512).long().to(device)\n}\nonnx_path = "bert_kor_base.onnx"\ntorch.onnx.export(bert, (dummy_input[\'input_ids\'], dummy_input[\'attention_mask\'], dummy_input[\'token_type_ids\']), onnx_path, input_names=[\'input_ids\', \'attention_mask\', \'token_type_ids\'], output_names=[\'output\'], dynamic_axes={\'input_ids\': {0: \'batch_size\'}, \'attention_mask\': {0: \'batch_size\'}, \'token_type_ids\': {0: \'batch_size\'}})\n\n# Triton Server 설정 파일 생성\ntriton_config = f"""\nname: "bert_kor_base"\nplatform: "onnxruntime_onnx"\n\nmax_batch_size: 1\ninput [\n  {{\n    name: "input_ids"\n    data_type: TYPE_INT64\n    dims: [512]\n  }},\n  {{\n    name: "attention_mask"\n    data_type: TYPE_INT64\n    dims: [512]\n  }},\n  {{\n    name: "token_type_ids"\n    data_type: TYPE_INT64\n    dims: [512]\n  }}\n]

In [14]:
import torch.onnx

# 예제 모델
example_model = CustomBertModel(CHECKPOINT_NAME)
example_model.load_state_dict(torch.load('bert-kor-base.pth'))
example_model.eval()

# 예제 입력 데이터 (가변적인 배치 크기를 처리하기 위해 None 사용)
example_inputs = {
    'input_ids': torch.zeros((8,512), dtype=torch.long),
    'attention_mask': torch.ones((8,512), dtype=torch.long),
    'token_type_ids': torch.zeros((8,512), dtype=torch.long)
}

# ONNX로 모델 변환
torch.onnx.export(
    example_model,
    (example_inputs['input_ids'], example_inputs['attention_mask'], example_inputs['token_type_ids']),
    'bert-kor-base.onnx',
    verbose=True
)

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertTokenizerFast, BertModel
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig
from transformers import BertTokenizerFast
from Korpora import Korpora
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# 한글 자연어 처리 데이터셋
from Korpora import Korpora

# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

corpus = Korpora.load("nsmc")
import pandas as pd

train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))
train = train.loc[train['length'] > 5]
# 전체 데이터셋 크기가 커서 1000개의 문장을 샘플링 합니다.
train = train.sample(1000)
test = test.loc[test['length'] > 5]
test = test.sample(500)

CHECKPOINT_NAME = 'kykim/bert-kor-base'

class TokenDataset(Dataset):
    def __init__(self, dataframe, tokenizer_pretrained):
        self.data = dataframe        
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        tokens = self.tokenizer(
            sentence,
            return_tensors='pt',
            truncation=True,
            padding='max_length',
            add_special_tokens=True
        )

        input_ids = tokens['input_ids'].squeeze(0)
        attention_mask = tokens['attention_mask'].squeeze(0)
        token_type_ids = torch.zeros_like(attention_mask)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

tokenizer_pretrained = CHECKPOINT_NAME
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)

tokenizer_bert = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model_bert = BertModel.from_pretrained("kykim/bert-kor-base")

inputs, labels = next(iter(train_loader))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

config = BertConfig.from_pretrained(CHECKPOINT_NAME)

labels

model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)

output = model_bert(**inputs)
output.keys()
output['last_hidden_state'].shape, output['pooler_output'].shape
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])

pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

fc = nn.Linear(768, 2)

fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))

import time

class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        super(CustomBertModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_pretrained)
        self.dr = nn.Dropout(p=dropout_rate)
        self.fc = nn.Linear(768, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = output['last_hidden_state']
        x = self.dr(last_hidden_state[:, 0, :])
        x = self.fc(x)
        return x

bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

from tqdm import tqdm

def model_train(model, data_loader, loss_fn, optimizer, device):
    model.train()
    running_loss = 0
    corr = 0
    counts = 0
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    
    for idx, (inputs, labels) in enumerate(prograss_bar):
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        output = model(**inputs)
        
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        
        _, pred = output.max(dim=1)
        corr += pred.eq(labels).sum().item()
        counts += len(labels)
        
        running_loss += loss.item() * labels.size(0)
        
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        
    acc = corr / len(data_loader.dataset)
    return running_loss / len(data_loader.dataset), acc

def model_evaluate(model, data_loader, loss_fn, device):
    model.eval()
    with torch.no_grad():
        corr = 0
        running_loss = 0
        for inputs, labels in data_loader:
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            
            output = model(**inputs)
            
            _, pred = output.max(dim=1)
            
            corr += torch.sum(pred.eq(labels)).item()
            
            running_loss += loss_fn(output, labels).item() * labels.size(0)
        
        acc = corr / len(data_loader.dataset)
        return running_loss / len(data_loader.dataset), acc

import torch.onnx

num_epochs = 100
model_name = 'bert-kor-base'
min_loss = np.inf

for epoch in range(num_epochs):
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)   
    
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}.pth')
        #torch.onnx.export(bert, (inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids']), 'bert-kor-base.onnx', verbose=True)
        # 예제 입력 생성 (배치 크기 1로 가정)
        example_input = {
            'input_ids': torch.zeros([1, 512], dtype=torch.long).to(device),
            'attention_mask': torch.zeros([1, 512], dtype=torch.long).to(device),
            'token_type_ids': torch.zeros([1, 512], dtype=torch.long).to(device),
             }

# ONNX로 변환
        onnx_path = 'bert_kor_base.onnx'
        torch.onnx.export(
              bert,
              args=(example_input['input_ids'], example_input['attention_mask'], example_input['token_type_ids']),
              f=onnx_path,
              input_names=['input_ids', 'attention_mask', 'token_type_ids'],
              output_names=['output'],
              dynamic_axes={'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'token_type_ids': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
              opset_version=11,  # ONNX 버전
              do_constant_folding=True,  # 상수 접기
              verbose=True,
              )
    print(" ") 
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

bert.load_state_dict(torch.load(f'{model_name}.pth'))


class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels
        
    def predict(self, sentence):
        tokens = self.tokenizer(
            sentence,
            return_tensors='pt',
            truncation=True,
            padding='max_length',
            add_special_tokens=True
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

labels = {
    0: '부정 리뷰 입니다.', 
    1: '긍정 리뷰 입니다.'
}

predictor = CustomPredictor(bert, tokenizer, labels)

def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)

predict_sentence(predictor)


import torch.onnx

# 모델을 evaluation mode로 설정
#bert.eval()



# Triton Server config.pbtxt 파일 작성
config_path = 'config.pbtxt'
with open(config_path, 'w') as f:
    f.write("""
name: "bert_kor_base"
platform: "onnxruntime_onnx"

max_batch_size: 8

input [
  {
    name: "input_ids"
    data_type: TYPE_INT64
    dims: [ -1, 512 ]
  },
  {
    name: "attention_mask"
    data_type: TYPE_INT64
    dims: [ -1, 512 ]
  },
  {
    name: "token_type_ids"
    data_type: TYPE_INT64
    dims: [ -1, 512 ]
  }
]

output [
  {
    name: "output"
    data_type: TYPE_FP32
    dims: [ -1, 2 ]
  }
]

dynamic_batching {
  preferred_batch_size: [ 1, 2, 4, 8 ]
  max_queue_delay_microseconds: 1000
  default_priority: 0
}

instance_group [
  {
    count: 1
    kind: KIND_GPU
  }
]
""")



사용 디바이스: cuda:1

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /home/eternal/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed 

training loss: 5.30132, training accuracy: 0.63500: 100%|█| 125/125 [00:14<00:00


[INFO] val_loss has been improved from inf to 0.42087. Saving Model!
 
epoch 01, loss: 0.66266, acc: 0.63500, val_loss: 0.42087, val_accuracy: 0.80800


training loss: 3.43187, training accuracy: 0.81800: 100%|█| 125/125 [00:15<00:00


 
epoch 02, loss: 0.42898, acc: 0.81800, val_loss: 0.47835, val_accuracy: 0.77600


...

 
epoch 98, loss: 0.00065, acc: 1.00000, val_loss: 1.16575, val_accuracy: 0.82600


training loss: 0.00135, training accuracy: 1.00000: 100%|█| 125/125 [00:14<00:00


 
epoch 99, loss: 0.00017, acc: 1.00000, val_loss: 1.16953, val_accuracy: 0.82600


training loss: 0.00128, training accuracy: 1.00000: 100%|█| 125/125 [00:13<00:00


 
epoch 100, loss: 0.00016, acc: 1.00000, val_loss: 1.17725, val_accuracy: 0.82800
문장을 입력해 주세요: ㅇㄴㄹ
[부정 리뷰 입니다.]
확률은: 87.539% 입니다.


In [7]:
import torch
from transformers import BertModel

# 모델 초기화 및 가중치 로드
model = CustomBertModel("kykim/bert-kor-base")
model.load_state_dict(torch.load('bert-kor-base.pth'))
model.to(device)
model.eval()

# ONNX 파일로 내보내기 위한 더미 입력 생성
# 이 부분은 모델의 실제 입력과 일치해야 합니다.
# 예를 들어, 모델이 128 토큰 길이의 입력을 받는다고 가정하면:
input_ids = torch.randint(0, 20000, (1, 128)) # 모델에 맞는 임의의 토큰 ID
attention_mask = torch.ones(1, 128)           # 모든 토큰이 "real"임을 나타내는 마스크
token_type_ids = torch.zeros(1, 128)          # 단일 문장 입력을 나타내는 토큰 타입 ID
'''
# input_ids, attention_mask, token_type_ids를 Long 타입으로 변환
input_ids = input_ids.long() if input_ids.dtype != torch.long else input_ids
attention_mask = attention_mask.long() if attention_mask.dtype != torch.long else attention_mask
token_type_ids = token_type_ids.long() if token_type_ids.dtype != torch.long else token_type_ids
'''

# 모든 더미 입력을 GPU로 이동 (GPU를 사용하는 경우)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
token_type_ids = token_type_ids.to(device)

# 모델과 더미 입력을 ONNX로 내보내기
torch.onnx.export(model, 
                  (input_ids, attention_mask, token_type_ids), 
                  "bert_kor_base.onnx",
                  export_params=True, 
                  opset_version=11, 
                  do_constant_folding=True,
                  input_names=['input_ids', 'attention_mask', 'token_type_ids'],
                  output_names=['output'],
                  dynamic_axes={'input_ids': {0: 'batch_size'},
                                'attention_mask': {0: 'batch_size'},
                                'token_type_ids': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})



NameError: name 'input_ids' is not defined

In [8]:
import torch
from transformers import BertModel

# 모델 초기화 및 가중치 로드
model = CustomBertModel("kykim/bert-kor-base")
model.load_state_dict(torch.load('bert-kor-base.pth'))
model.to(device)
model.eval()

# ONNX 파일로 내보내기 위한 더미 입력 생성
# 모델이 128 토큰 길이의 입력을 받는다고 가정
input_ids = torch.randint(0, 20000, (1, 512)) # 모델에 맞는 임의의 토큰 ID
attention_mask = torch.ones(1, 512)           # 모든 토큰이 "real"임을 나타내는 마스크
token_type_ids = torch.zeros(1, 512)          # 단일 문장 입력을 나타내는 토큰 타입 ID

# 모든 더미 입력을 Long 타입으로 변환하고 GPU로 이동 (GPU를 사용하는 경우)
input_ids = input_ids.long().to(device)
attention_mask = attention_mask.long().to(device)
token_type_ids = token_type_ids.long().to(device)

# 모델과 더미 입력을 ONNX로 내보내기
torch.onnx.export(model, 
                  (input_ids, attention_mask, token_type_ids), 
                  "bert_kor_base.onnx",
                  export_params=True, 
                  opset_version=11, 
                  do_constant_folding=True,
                  input_names=['input_ids', 'attention_mask', 'token_type_ids'],
                  output_names=['output'],
                  dynamic_axes={'input_ids': {0: 'batch_size'},
                                'attention_mask': {0: 'batch_size'},
                                'token_type_ids': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})


In [6]:
import onnxruntime as ort
import numpy as np

# ONNX 모델 로드
session = ort.InferenceSession("bert_kor_base.onnx")

# 모델 입력을 위한 더미 데이터 생성
# 실제 데이터와 동일한 형식을 사용하세요.
input_ids = np.random.randint(0, 20000, size=(1, 128), dtype=np.int64)
attention_mask = np.ones((1, 128), dtype=np.int64)
token_type_ids = np.zeros((1, 128), dtype=np.int64)

# 모델 실행
inputs = {
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    "token_type_ids": token_type_ids
}
outputs = session.run(None, inputs)

# 출력 확인
print(outputs)


[array([[-0.38527295,  1.0995104 ]], dtype=float32)]


In [7]:
import torch.nn.functional as F

logits = np.array([-0.50374013, 1.2210462])
probabilities = F.softmax(torch.tensor(logits), dim=0).numpy()
print(probabilities)

[0.15125568 0.84874432]


클라이언트 파트

In [ ]:
import tritonclient.http as httpclient
import numpy as np
from transformers import BertTokenizer

# 서버 URL 설정
TRITON_SERVER_URL = "localhost:8000"

# 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')

# 클라이언트 객체 생성
client = httpclient.InferenceServerClient(url=TRITON_SERVER_URL)

# 분석할 텍스트
#text = "여기에 분석할 텍스트를 입력하세요"
# 분석할 텍스트 입력 받기
text = input("분석할 텍스트를 입력하세요: ")


# 토크나이징 및 텐서 변환
inputs = tokenizer(text, return_tensors="np", max_length=128, truncation=True, padding='max_length')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
token_type_ids = inputs['token_type_ids']

# Triton 서버 추론 요청
triton_inputs = [
    httpclient.InferInput("input_ids", input_ids.shape, "INT64"),
    httpclient.InferInput("attention_mask", attention_mask.shape, "INT64"),
    httpclient.InferInput("token_type_ids", token_type_ids.shape, "INT64"),
]

triton_inputs[0].set_data_from_numpy(input_ids)
triton_inputs[1].set_data_from_numpy(attention_mask)
triton_inputs[2].set_data_from_numpy(token_type_ids)

response = client.infer("bert_kor_base", triton_inputs)

# 추론 결과
output = response.as_numpy("output")
prediction = np.argmax(output, axis=1)

print("긍정" if prediction[0] == 1 else "부정")


In [4]:
import tritonclient.http as httpclient
import numpy as np
import torch.nn.functional as F
from transformers import BertTokenizer
import torch


# 서버 URL 설정
TRITON_SERVER_URL = "localhost:8000"
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')
client = httpclient.InferenceServerClient(url=TRITON_SERVER_URL)

# 사용자 입력 받기
text = input("분석할 텍스트를 입력하세요: ")
inputs = tokenizer(text, return_tensors="np", max_length=128, truncation=True, padding='max_length')

# Triton 서버 추론 요청
triton_inputs = [
    httpclient.InferInput("input_ids", inputs['input_ids'].shape, "INT64"),
    httpclient.InferInput("attention_mask", inputs['attention_mask'].shape, "INT64"),
    httpclient.InferInput("token_type_ids", inputs['token_type_ids'].shape, "INT64"),
]

triton_inputs[0].set_data_from_numpy(inputs['input_ids'])
triton_inputs[1].set_data_from_numpy(inputs['attention_mask'])
triton_inputs[2].set_data_from_numpy(inputs['token_type_ids'])

response = client.infer("bert_kor_base", triton_inputs)
output = response.as_numpy("output")

# 확률 계산 및 출력
probabilities = F.softmax(torch.tensor(output), dim=1).numpy()
predicted_class = np.argmax(probabilities, axis=1)
confidence_score = probabilities[0, predicted_class[0]] * 100

print(f"예측된 클래스: {'긍정' if predicted_class[0] == 1 else '부정'}, 확률: {confidence_score:.2f}%")


분석할 텍스트를 입력하세요: asd
예측된 클래스: 부정, 확률: 85.06%


In [ ]:
#perf_analyzer -m your_model_name --shape INPUT_NAME:DIM1,DIM2
perf_analyzer -m bert_kor_base --shape "input_ids:1,128" --shape "attention_mask:1,128" --shape "token_type_ids:1,128"
sudo docker run --rm --gpus=1 -p 8000:8000 -p 8001:8001 -p 8002:8002 -v /home/eternal/testpy:/models nvcr.io/nvidia/tritonserver:23.10-py3 tritonserver --model-repository=/models
sudo docker run -it --rm --net=host nvcr.io/nvidia/tritonserver:23.10-py3-sdk

In [6]:
import time
import requests

# 추론 서버 URL
url = 'http://localhost:8000/v2/models/your_model_name/infer'

# 요청 데이터
data = {'input': 'your_input_data'}

# 요청 반복 횟수
num_requests = 1000

# 쓰루풋 측정 시작
start_time = time.time()

for _ in range(num_requests):
    response = requests.post(url, json=data)

end_time = time.time()

# 총 소요 시간과 쓰루풋 계산
total_time = end_time - start_time
throughput = num_requests / total_time

print(f"Total time: {total_time} seconds")
print(f"Throughput: {throughput} requests/second")


Total time: 0.43674731254577637 seconds
Throughput: 2289.6534707244205 requests/second


In [ ]:
*** Measurement Settings ***
  Batch size: 1
  Service Kind: Triton
  Using "time_windows" mode for stabilization
  Measurement window: 5000 msec
  Using synchronous calls for inference
  Stabilizing using average latency

Request concurrency: 1
  Client: 
    Request count: 9907
    Throughput: 550.257 infer/sec
    Avg latency: 1816 usec (standard deviation 120 usec)
    p50 latency: 1796 usec
    p90 latency: 1881 usec
    p95 latency: 1936 usec
    p99 latency: 2107 usec
    Avg HTTP time: 1815 usec (send/recv 12 usec + response wait 1803 usec)
  Server: 
    Inference count: 9907
    Execution count: 9907
    Successful request count: 9907
    Avg request latency: 1757 usec (overhead 4 usec + queue 4 usec + compute input 7 usec + compute infer 1739 usec + compute output 2 usec)

In [21]:
import tritonclient.http as httpclient
import numpy as np
import time

# Triton 서버 설정
triton_url = "localhost:8000"  # Triton 서버 URL
model_name = "bert_kor_base"

# 클라이언트 생성
client = httpclient.InferenceServerClient(url=triton_url)

# 요청을 위한 입력 데이터 생성
input_ids = np.random.randint(0, 20000, (1, 128), dtype=np.int64)
attention_mask = np.ones((1, 128), dtype=np.int64)
token_type_ids = np.zeros((1, 128), dtype=np.int64)

# 입력 데이터를 Triton에 맞게 포맷팅
inputs = [
    httpclient.InferInput("input_ids", [1, 128], "INT64"),
    httpclient.InferInput("attention_mask", [1, 128], "INT64"),
    httpclient.InferInput("token_type_ids", [1, 128], "INT64")
]
inputs[0].set_data_from_numpy(input_ids)
inputs[1].set_data_from_numpy(attention_mask)
inputs[2].set_data_from_numpy(token_type_ids)

# 쓰루풋 측정을 위한 시간 측정 시작
start_time = time.time()

# 요청 반복
num_requests = 100
for _ in range(num_requests):
    response = client.infer(model_name, inputs)

# 쓰루풋 계산
elapsed_time = time.time() - start_time
throughput = num_requests / elapsed_time

print(f"쓰루풋: {throughput} 요청/초")


쓰루풋: 24.09597661400463 요청/초


In [ ]:
import time
import requests

# 추론 서버 URL
url = 'http://localhost:8000/v2/models/your_model_name/infer'

# 요청 데이터
data = {'input': 'your_input_data'}

# 요청 반복 횟수
num_requests = 1000

# 쓰루풋 측정 시작
start_time = time.time()

for _ in range(num_requests):
    response = requests.post(url, json=data)

end_time = time.time()

# 총 소요 시간과 쓰루풋 계산
total_time = end_time - start_time
throughput = num_requests / total_time

print(f"Total time: {total_time} seconds")
print(f"Throughput: {throughput} requests/second")
